# <center> <font color='blue'> PROJETO: Prevendo o Nível de Satisfação dos Clientes do Santander

![title](imagens/projeto.png)

<justify> Neste projeto iremos utilizar dados dos clientes do santander para avaliar o nível de satisfação dos mesmos, tais informações serão de grande utilizade para avaliação de campanhas capazes de aumentar o nível de satisfação de clientes insatisfeitos e manter a alta satisfação dos demais. O principal desafio a nossa frente é apresentar um modelo com acurácia superior a 70% para identificar clientes insatisfeitos.

## <center> <font color='blue'> FERRAMENTAS  

In [ ]:
#Importando as bibliotecas a serem utilizadas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from imblearn.over_sampling import SMOTE
#from imblearn.under_sampling import 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import ConfusionMatrixDisplay  
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.utils import compute_sample_weight
import warnings
from scipy import stats
from minhas_funcoes import *

In [ ]:
warnings.filterwarnings("ignore")
plt.rcParams.update({'font.size': 14})

## <center> <font color='blue'> ANÁLISE EXPLORATÓRIA DOS DADOS

In [ ]:
#Carregando os dados
#iremos carregar os dados em um df para realizar análise exploratória

df = pd.read_csv("datasets/train.csv")

In [ ]:
#O dataset é constituido por 371 variáveis preditoras e 76020 observações

df.shape

In [ ]:
#Cinco primeiras observações

df.head()

<justify>O dataset apresenta um elevado número de variváveis (370), o que pode ser levemente problemático...

In [ ]:
#A variável ID presente no dataset não apresenta nenhuma informação relevante para a análise de satsfação, dito isto 
#iremos excluir
df = df.iloc[:,1:]
df.head()

In [ ]:
df.describe()

In [ ]:
#Distribuição da variável alvo
df["TARGET"].value_counts().sort_index()

A variável alvo em nosso estudo encontra se extremamente desbalanceada

In [ ]:
#Graficamente
plt.subplots(figsize=(8,8))
plt.hist(df["TARGET"], facecolor='b', alpha=1)
plt.xlabel('Satisfação')
plt.ylabel('Observações')
plt.title('Histograma de satisfação dos clientes Santander')
plt.xlim(0, 1)
plt.ylim(0, 80000)
plt.grid(False)
plt.show()

O nível de satisfação é representado pelos valores numéricos 0 e 1, onde 0 representa os clientes satisfeitos e 1 os clientes insatisfeitos com os serviços do Santander.

In [ ]:
X = df.iloc[:,0:271]
y = pd.DataFrame(df["TARGET"])

In [ ]:
#Nome das colunas do dataframe
coluna = X.columns

#localizando indices dos outliers
indices_out = tratamento_outliers(X, coluna)

#removendo outliers de acordo com o Z-escore
x_s_out = X.drop(indices_out)
y_s_out = y.drop(indices_out)

In [ ]:
#Iremos separar os dados entre treino e teste para assim balancearmos as classes alvo no dataset de treino através do SMOTE 
#(Synthetic Minority Oversampling Technique) e evitarmos problemas posteriores.

split = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)

for train_index, test_index in split.split(x_s_out, y_s_out):
    X_treino, X_teste = X.loc[train_index], X.loc[test_index]
    y_treino, y_teste = y.loc[train_index], y.loc[test_index]

In [ ]:
#escalonando as variáveis de treino

escalonador = StandardScaler()
X_treino_esc = escalonador.fit_transform(X_treino)

In [ ]:
peso_classe_balanceado = compute_sample_weight("balanced", y_treino)
peso_classe_ = compute_sample_weight({0:1, 1:30}, y_treino)

## <center> <font color='blue'> TRABALHANDO A QUESTÃO DA DIMENSIONALIDADE

In [ ]:
treino_pca(random_forest, X_treino, X_teste, y_treino, y_teste, taxa_de_variancia=10)

In [ ]:
diagrama_variancia(X_treino, 10)

## <center> <font color='blue'> TREINANDO MODELO RANDOM FOREST BASE

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
random_forest.fit(X_treino, np.ravel(y_treino));

In [ ]:
plot_multiple_cm(random_forest, X_treino, X_teste, y_treino, y_teste)

### <center> <font color='blue'> GRID search


In [ ]:
param_grid_forest = [{'bootstrap': [False],
                      'criterion': ['entropy'],
                      'n_estimators': [300, 500], 
                      'max_features': [3, 6, 9],
                      'max_depth': [4, 6, 8, 10]}]


grid_search_forest = GridSearchCV(random_forest, param_grid_forest, cv = 3, scoring = 'accuracy',
                                  return_train_score = True, n_jobs=-1)

grid_search_forest.fit(X_treino, np.ravel(y_treino), sample_weight=peso_classe)

In [ ]:
best_random_forest = RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=4,
                       max_features=3, n_estimators=300, n_jobs=-1,
                       random_state=42)

best_random_forest.fit(X_treino, np.ravel(y_treino), sample_weight=peso_classe)

In [ ]:
plot_multiple_cm(best_random_forest, X_treino, X_teste, y_treino, y_teste)
#plt.savefig("cm.jpeg")

## <center> <font color='blue'> TREINANDO MODELO XGBOOST

In [ ]:
xgb_base = xgb.XGBClassifier(use_label_encoder=False, random_state=42, eval_metric='logloss') #scale_pos_weight=24


In [ ]:
xgb_base.fit(X_treino,np.ravel(y_treino))

In [ ]:
plot_multiple_cm(xgb_base, X_treino, X_teste, y_treino, y_teste)

### <center> <font color='blue'> GRID search

In [ ]:
params={'learning_rate' : [0.25,0.30,0.35], 'max_depth' : [4, 6, 8, 10],
 'gamma': [ 0.0, 0.3, 0.5 ],      
 'eval_metric': ['error']
}

grid_search_xgb = GridSearchCV(xgb_base,param_grid=params,n_jobs=-1,cv=3,verbose=2)

In [ ]:
grid_search_xgb.fit(X_treino,np.ravel(y_treino))

In [ ]:
xgb_best  = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
               eval_metric='logloss', gamma=0.0, gpu_id=-1, importance_type=None,
               interaction_constraints='', learning_rate=0.25, max_delta_step=0,
               max_depth=4, min_child_weight=1,
               monotone_constraints='()', n_estimators=100, n_jobs=8,
               num_parallel_tree=1, predictor='auto', random_state=42,
               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
               tree_method='exact', use_label_encoder=False,
               validate_parameters=1, verbosity=None)

In [ ]:
xgb_best.fit(X_treino, y_treino, sample_weight=peso_classe2)

In [ ]:
plot_multiple_cm(xgb_best, X_treino, X_teste, y_treino, y_teste)